In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os
import torch
import torchvision
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim

# List dataset files
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [4]:
class AgeDataset(Dataset):
    def __init__(self, data_path, annot_path, train=True):
        self.data_path = data_path
        self.annot_path = annot_path
        self.train = train
        self.df = pd.read_csv(annot_path)
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_path = os.path.join(self.data_path, self.df.iloc[index]['file_id'])
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        if self.train:
            age = self.df.iloc[index]['age']
            return image, age
        return image

train_dataset = AgeDataset('/kaggle/input/smai-24-age-prediction/content/faces_dataset/train',
                           '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train.csv', train=True)
test_dataset = AgeDataset('/kaggle/input/smai-24-age-prediction/content/faces_dataset/test',
                          '/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv', train=False)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Using a pretrained ResNet
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False  # Freeze all layers

# Replace the final layer to suit our output (age prediction)
model.fc = nn.Linear(model.fc.in_features, 1)
model = model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, ages in tqdm(train_loader):
        images = images.to(device)
        ages = ages.to(device).float().view(-1, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, ages)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 160MB/s] 
100%|██████████| 334/334 [02:45<00:00,  2.02it/s]


Epoch 1, Loss: 348.3560644366784


100%|██████████| 334/334 [01:38<00:00,  3.41it/s]


Epoch 2, Loss: 236.0858169829774


100%|██████████| 334/334 [01:36<00:00,  3.48it/s]


...

Epoch 8, Loss: 167.3916147197792


100%|██████████| 334/334 [01:36<00:00,  3.46it/s]


Epoch 9, Loss: 165.04895311915232


100%|██████████| 334/334 [01:37<00:00,  3.44it/s]

Epoch 10, Loss: 162.94782028655092


In [6]:
@torch.no_grad()
def predict(loader, model):
    model.eval()
    predictions = []
    for images in tqdm(loader):
        images = images.to(device)
        outputs = model(images)
        predictions.extend(outputs.view(-1).cpu().detach().numpy())

    return predictions

preds = predict(test_loader, model)

submit = pd.read_csv('/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv')
submit['age'] = preds
submit.to_csv('baseline.csv', index=False)


100%|██████████| 31/31 [00:19<00:00,  1.62it/s]
